# 03. Data Processing and Feature Engineering

## Objective
In this notebook, we transform our raw data into a clean, unified dataset ready for analysis. Our goal is to calculate a **"Demand Score"** for each neighborhood.

## Steps
1.  **Load Data**: Reload our raw datasets (Chargers, Vehicles, Income, Neighborhoods).
2.  **Clean & Standardize**: Ensure all datasets use compatible naming conventions (e.g., Neighborhood Codes).
3.  **Spatial Join**: Determine which neighborhood each existing charger belongs to.
4.  **Aggregate Data**: Calculate total chargers, average income, and vehicle metrics per neighborhood.
5.  **Calculate Demand Score**: Create a unified metric to identify high-potential areas.
6.  **Export**: Save the processed data for the next phase (Optimization).

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os

# 1. Dynamic Path Setup (so this runs on any machine)
def get_data_dir():
    # Check common paths relative to the notebook
    possible_paths = ['../data/raw', 'data/raw', '../../data/raw']
    for path in possible_paths:
        if os.path.exists(path):
            return path
    # Fallback to absolute path logic if needed
    return os.path.join(os.getcwd(), 'data', 'raw')

RAW_DATA_DIR = get_data_dir()
print(f"Loading data from: {os.path.abspath(RAW_DATA_DIR)}")

FILE_GEOMETRY = os.path.join(RAW_DATA_DIR, '0301100100_UNITATS_ADM_POLIGONS.json')
FILE_CHARGERS = os.path.join(RAW_DATA_DIR, '2023_2T_Punts_Recarrega_Vehicle_Electric.json')
FILE_VEHICLES = os.path.join(RAW_DATA_DIR, '2024_parc_vehicles_tipus_propulsio.csv')
FILE_INCOME = os.path.join(RAW_DATA_DIR, '2022_renda_disponible_llars_per_persona.csv')

Loading data from: c:\Users\nicoh\OneDrive\Documentos\WIP-EV_hub_optimization\data\raw


## 2. Data Loading
We use the robust loading logic we developed in the previous step.

In [2]:
# Load Neighborhoods (Geometry)
gdf_barris = gpd.read_file(FILE_GEOMETRY)
# Ensure CRS is standard WGS84 (Latitude/Longitude)
if gdf_barris.crs != 'EPSG:4326':
    gdf_barris = gdf_barris.to_crs('EPSG:4326')

print(f"Loaded {len(gdf_barris)} administrative units.")

# Load Chargers (JSON -> GeoDataFrame)
try:
    gdf_chargers = gpd.read_file(FILE_CHARGERS)
except:
    # Fallback for flat JSON
    df_temp = pd.read_json(FILE_CHARGERS)
    gdf_chargers = gpd.GeoDataFrame(
        df_temp,
        geometry=gpd.points_from_xy(df_temp.Station_lng, df_temp.Station_lat),
        crs="EPSG:4326"
    )
print(f"Loaded {len(gdf_chargers)} chargers.")

# Load Income
df_income = pd.read_csv(FILE_INCOME)
print(f"Loaded income data: {len(df_income)} rows.")

# Load Vehicles
df_vehicles = pd.read_csv(FILE_VEHICLES)
print(f"Loaded vehicle data: {len(df_vehicles)} rows.")

Loaded 1501 administrative units.
Loaded 393 chargers.
Loaded income data: 1068 rows.
Loaded vehicle data: 5787 rows.


## 3. Data Cleaning & Standardization
We need to filter the geometry to only keep Neighborhoods ('BARRI') and then standardize the ID.

In [3]:
# Inspect column names to ensure we pick the right ones
# print("Geometry Columns:", gdf_barris.columns)

In [4]:
# Standardize 'Codi_Barri' to Integer

# Neighborhoods (Geometry)
# The file contains multiple administrative levels (District, Neighborhood, etc.)
# We need to filter only for "BARRI"
if 'TIPUS_UA' in gdf_barris.columns:
    print(f"Filtering geometry for 'BARRI' levels. Original size: {len(gdf_barris)}")
    gdf_barris = gdf_barris[gdf_barris['TIPUS_UA'] == 'BARRI'].copy()
    print(f"Filtered size: {len(gdf_barris)}")

# Identify the correct column for Neighborhood Code
col_barri_geo = 'BARRI'
if col_barri_geo not in gdf_barris.columns:
    candidates = [c for c in gdf_barris.columns if 'BARRI' in c.upper()]
    col_barri_geo = candidates[0] if candidates else 'CODI_UA'

print(f"Using '{col_barri_geo}' as Geometry Neighborhood Code column.")

# Clean the column (handle "1" vs 1 vs "01")
gdf_barris['Barri_ID'] = gdf_barris[col_barri_geo].astype(str).str.strip().astype(int)

# Income
df_income['Barri_ID'] = df_income['Codi_Barri'].astype(int)

# Vehicles
# 'Codi_Barri' may contain non-numeric values like 'NC' (No Consta)
# We convert them to NaN, then drop those rows, then convert to int.
df_vehicles['Barri_ID'] = pd.to_numeric(df_vehicles['Codi_Barri'], errors='coerce')
print(f"Dropping {df_vehicles['Barri_ID'].isna().sum()} rows with invalid Barri ID in vehicles data.")
df_vehicles = df_vehicles.dropna(subset=['Barri_ID'])
df_vehicles['Barri_ID'] = df_vehicles['Barri_ID'].astype(int)

print("Standardization complete. Using 'Barri_ID' as the common key.")

Filtering geometry for 'BARRI' levels. Original size: 1501
Filtered size: 73
Using 'BARRI' as Geometry Neighborhood Code column.
Dropping 3 rows with invalid Barri ID in vehicles data.
Standardization complete. Using 'Barri_ID' as the common key.


## 4. Aggregating Socio-Economic Data
We need one row per Neighborhood.

### 4.1 Income
We'll take the mean income per neighborhood.

In [5]:
df_income_agg = df_income.groupby('Barri_ID')['Import_Euros'].mean().reset_index()
df_income_agg.rename(columns={'Import_Euros': 'Avg_Income'}, inplace=True)
display(df_income_agg.head())

,Barri_ID,Avg_Income
0,1,14221.142857
1,2,18668.111111
2,3,17965.909091
3,4,19655.153846
4,5,24163.200000


### 4.2 Vehicles
We now have a file broken down by propulsion type. We will calculate:
1. **Total Vehicles**: General traffic volume.
2. **EV Count**: Count of 'Elèctrica' and 'Híbrid'.

In [6]:
# Calculate Total Vehicles AND EV Counts by Neighborhood
if 'Nombre' in df_vehicles.columns:
    print("Found 'Nombre' column. Calculating Total Vehicles and EVs...")
    
    # 1. Total Vehicles
    df_vehicles_agg = df_vehicles.groupby('Barri_ID')['Nombre'].sum().reset_index()
    df_vehicles_agg.rename(columns={'Nombre': 'Total_Vehicles'}, inplace=True)
    
    # 2. Electric Vehicles (Elèctrica + Híbrid)
    # We filter for 'Elèctrica' and 'Híbrid'
    target_propulsion = ['Elèctrica', 'Híbrid']
    df_evs = df_vehicles[df_vehicles['Tipus_Propulsio'].isin(target_propulsion)]
    
    df_ev_agg = df_evs.groupby('Barri_ID')['Nombre'].sum().reset_index()
    df_ev_agg.rename(columns={'Nombre': 'EV_Count'}, inplace=True)
    
    # Merge EV count into main aggregate
    df_vehicles_agg = df_vehicles_agg.merge(df_ev_agg, on='Barri_ID', how='left')
    df_vehicles_agg['EV_Count'] = df_vehicles_agg['EV_Count'].fillna(0)
    
else:
    print("Error: 'Nombre' column not found in vehicle dataset!")
    # Fallback
    df_vehicles_agg = df_vehicles.groupby('Barri_ID').size().reset_index(name='Total_Vehicles')
    df_vehicles_agg['EV_Count'] = 0

display(df_vehicles_agg.head())

Found 'Nombre' column. Calculating Total Vehicles and EVs...


,Barri_ID,Total_Vehicles,EV_Count
0,1,11725,897
1,2,10722,525
2,3,9260,2378
3,4,13947,4620
4,5,14321,1206


## 5. Spatial Join: Mapping Chargers to Neighborhoods
We have charger points (Lat/Lon). We need to count how many chargers are inside each Neighborhood polygon.

In [7]:
# Perform Spatial Join (Left Join: Chargers -> Neighborhoods)
# We strictly use 'intersects' or 'within'
gdf_joined = gpd.sjoin(gdf_chargers, gdf_barris, how="left", predicate="within")

# Count chargers per Barri
chargers_per_barri = gdf_joined.groupby('Barri_ID').size().reset_index(name='Charger_Count')
display(chargers_per_barri.head())

,Barri_ID,Charger_Count
0,1.0,13
1,2.0,10
2,3.0,17
3,4.0,2
4,5.0,67


## 6. Creating the Master Dataset (Barrios with Demand)
Now we merge everything into the master `gdf_barris` dataframe.

In [8]:
# 1. Start with Neighborhood Geometry
master_df = gdf_barris[['Barri_ID', 'NOM', 'geometry']].copy() # Selecting key columns

# 2. Merge Income
master_df = master_df.merge(df_income_agg, on='Barri_ID', how='left')

# 3. Merge Vehicles
master_df = master_df.merge(df_vehicles_agg, on='Barri_ID', how='left')

# 4. Merge Charger Counts
master_df = master_df.merge(chargers_per_barri, on='Barri_ID', how='left')

# Fill NaNs (if a barrio has no chargers, count is 0)
master_df['Charger_Count'] = master_df['Charger_Count'].fillna(0)
# Fill missing income/vehicles with median (to avoid breaking the model)
master_df['Avg_Income'] = master_df['Avg_Income'].fillna(master_df['Avg_Income'].median())
master_df['Total_Vehicles'] = master_df['Total_Vehicles'].fillna(master_df['Total_Vehicles'].median())
master_df['EV_Count'] = master_df['EV_Count'].fillna(0)

display(master_df.head())

,Barri_ID,NOM,geometry,Avg_Income,Total_Vehicles,EV_Count,Charger_Count
0,1,el Raval,"POLYGON ((2.16471 41.38593, 2.16401 41.38540, ...",14221.142857,11725,897,13.0
1,2,el Barri Gòtic,"POLYGON ((2.17701 41.38525, 2.17658 41.38558, ...",18668.111111,10722,525,10.0
2,3,la Barceloneta,"POLYGON ((2.19623 41.38745, 2.19617 41.38746, ...",17965.909091,9260,2378,17.0
3,4,"Sant Pere, Santa Caterina i la Ribera","POLYGON ((2.18345 41.39061, 2.18238 41.39142, ...",19655.153846,13947,4620,2.0
4,5,el Fort Pienc,"POLYGON ((2.18353 41.39227, 2.18388 41.39253, ...",24163.200000,14321,1206,67.0


## 7. Calculate "Demand Score"
We create a composite score.

**New Logic (incorporating EVs):**
*   **EV Count** (50%): Strongest indicator. Areas with EVs *need* chargers right now.
*   **Family Income** (30%): Indicator of purchasing power for future EVs.
*   **Total Vehicles** (20%): Indicator of general traffic density and potential market size.

$$ Score = (Norm\_EVs \times 0.5) + (Norm\_Income \times 0.3) + (Norm\_TotalVehicles \times 0.2) $$

In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# Normalize Features
master_df['Norm_Income'] = scaler.fit_transform(master_df[['Avg_Income']])
master_df['Norm_Vehicles'] = scaler.fit_transform(master_df[['Total_Vehicles']])
master_df['Norm_EVs'] = scaler.fit_transform(master_df[['EV_Count']])

# Calculate Demand Score (0 to 100)
# Weight: 50% Existing EVs, 30% Income, 20% Total Traffic
master_df['Demand_Score'] = (
    master_df['Norm_EVs'] * 0.5 + 
    master_df['Norm_Income'] * 0.3 + 
    master_df['Norm_Vehicles'] * 0.2
) * 100

# Display top 5 neighborhoods by Demand
display(master_df[['NOM', 'Demand_Score', 'EV_Count', 'Total_Vehicles', 'Avg_Income']].sort_values(by='Demand_Score', ascending=False).head(10))

,NOM,Demand_Score,EV_Count,Total_Vehicles,Avg_Income
25,Sant Gervasi - Galvany,83.468167,3575,36060,36283.193548
18,les Corts,70.953194,3532,27575,29115.228571
6,la Dreta de l'Eixample,68.997667,3181,29640,29815.344828
3,"Sant Pere, Santa Caterina i la Ribera",65.551732,4620,13947,19655.153846
8,la Nova Esquerra de l'Eixample,59.018276,2647,28673,26255.950000
24,Sant Gervasi - la Bonanova,55.413504,1944,18094,36110.222222
11,la Marina del Prat Vermell,55.115824,4020,13594,16015.000000
7,l'Antiga Esquerra de l'Eixample,51.042672,2048,22448,28278.500000
22,Sarrià,49.332645,1517,16757,35446.375000
23,les Tres Torres,48.995498,1147,12680,41344.181818


## 8. Save Processed Data
We save this master dataset to `data/processed` for the optimization notebook.

In [10]:
PROCESSED_DIR = '../data/processed'
if not os.path.exists(PROCESSED_DIR):
    # Try absolute if relative fails (for robustness)
    PROCESSED_DIR = os.path.join(os.getcwd(), '..', 'data', 'processed')
    if not os.path.exists(PROCESSED_DIR):
         try:
             os.makedirs(PROCESSED_DIR)
         except:
             # last resort basic
             PROCESSED_DIR = 'data/processed'
             os.makedirs(PROCESSED_DIR, exist_ok=True)

OUT_FILE = os.path.join(PROCESSED_DIR, 'barrios_with_demand.geojson')

# Save as GeoJSON (preserves geometry)
master_df.to_file(OUT_FILE, driver='GeoJSON')

print(f"Success! Data saved to: {os.path.abspath(OUT_FILE)}")

Success! Data saved to: c:\Users\nicoh\OneDrive\Documentos\WIP-EV_hub_optimization\data\processed\barrios_with_demand.geojson
